<a href="https://colab.research.google.com/github/educatorsRlearners/hugging_face_course/blob/main/03_fine_tuning_a_pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 3. Fine-Tuning a pretrained model

It's all well and good to use toy data sets, but what about fine-tuning a pretrained model for your own? 

In this chapter we'll: 


*   prepare a large dataset from the Hub
*   use the ```Trainer``` API to fine-tune a model
*   use a custom training loop
*   leverage the  🤗 Accelerate library to run a custom training loop on a distributed setup



In [3]:
!pip install transformers datasets

## [Processing the data](https://huggingface.co/course/chapter3/2?fw=pt)

This is how we would train a sequence classifier on one batch using PyTorch: 

In [4]:
import torch 
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

batch = tokenizer(sequences, 
                  padding=True,
                  truncation=True,
                  return_tensors="pt")

batch["labels"] = torch.tensor([1,1])

optimizer = AdamW(model.parameters())

loss = model(**batch).loss
loss.backward()
optimizer.step()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Of course the example above is just that: an example. If we want to get decent results, we're going to need to train on a much larger dataset. Now where can we find one of those? 

### Loading a dataset from the Hub


In [7]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

If we want to inspect our data, we index it like a dictionary: 

In [8]:
raw_train_dataset = raw_datasets["train"]

raw_train_dataset[0]

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

What are the different labels for the dataset above? 

In [12]:
raw_train_dataset.features["label"]

ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None)

Now that we know what's in our dataset, we can start...

### Preprocessing a dataset

We've previously preprocessed single sentences but what about pairs of sentences? For instance, what if we want to train a model for if a sentence natrually follows on from the previous one, if questions are duplicates, or if there is plagiarism? 

To do so, we need to preporcess sentence pairs like this: 

In [24]:
tokenized_pairs = tokenizer("My name is Evan.", "I work for the AI Guild")

print(tokenized_pairs['input_ids'])
print(tokenized_pairs['token_type_ids'])
print(tokenized_pairs["attention_mask"])

[101, 2026, 2171, 2003, 9340, 1012, 102, 1045, 2147, 2005, 1996, 9932, 9054, 102]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


**Key Point**: ```token_type_ids```, (i.e., which tokens belong to which sequence) is only returned by models which are trained to handle multiple sentences. 

For example, BERT is trained on masked language tasks as well as sequencing (i.e., "Does this sentnece naturally follow on from the previous one?"). Distil-BERT, on the other hand, does not.

BTW, if we have several pairs, we can pass them like this: 

In [26]:
tokenized_groups = tokenizer(
    ["My name is Evan.", "I'm going to the movies."], #first sentences
    ["I work for the AI Guild.", "This song rocks!"], 
    padding=True
)

print(tokenized_groups['input_ids'])
print(tokenized_groups['token_type_ids'])
print(tokenized_groups["attention_mask"])

[[101, 2026, 2171, 2003, 9340, 1012, 102, 1045, 2147, 2005, 1996, 9932, 9054, 1012, 102], [101, 1045, 1005, 1049, 2183, 2000, 1996, 5691, 1012, 102, 2023, 2299, 5749, 999, 102]]
[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


Now while the above method works, it causes some issues in that it returns a dictionary so we'll probably end up running out of RAM unless we're using a toy dataset. 

To avoid this issue, we can use the ```Dataset.map()``` method so as to keep our data as an Apache Arrow file. 

In [28]:
def tokenize_function(example):
  return tokenizer(example["sentence1"], example["sentence2"],
                   truncation=True)

**Key Point**: we've omitted ```padding=True``` because padding is best done during batching because we don't need the length of the samples to match until they are ready to go into training; otherwise, we're just creating a massive dataset. 

Now we can apply our tokenize function on the dataset in one go like so: 

In [31]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d7bedce9e355b397.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-93ce23a30f1e95e1.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-1dbb17d82fd35b0a.arrow


In [32]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 1725
    })
})

### Dynamic Padding

Fixed padding is padding every input to a given length. 

Dynamic padding is padding every input to the max lenght of the longest input for a given batch. 

**NB**: DO NOT use dynamic padding on a TPU.

How do we do dynamic padding with the ```transformers``` library? 

Like so:

In [35]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

It really is just that simple. 

Now let's try it out with some samples from our training data.

In [44]:
samples = tokenized_datasets['train'][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

So, based on what we see above, we need to pad this batch to a max length of 67. 

In [46]:
batch = data_collator(samples)

{k:v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 67]),
 'input_ids': torch.Size([8, 67]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 67])}

## [Fine-tuning a model with the Trainer API](https://huggingface.co/course/chapter3/3?fw=pt)